# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'X-Linked Lymphoproliferative Syndrome'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = False

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Acute_Myeloid_Leukemia_(LAML)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LAML.sampleMap_LAML_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LAML.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['FISH_test_component',
 'FISH_test_component_percentage_value',
 '_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_LAML',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_miRNA_PANCAN',
 '_PANCAN_mirna_LAML']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['FISH_test_component', 'FISH_test_component_percentage_value', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_LAML', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_LAML', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'acute_myeloid_leukemia_calgb_cytogenetics_risk_category', 'age_at_initial_pathologic_diagnosis', 'atra_exposure', 'cum

In [19]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [20]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [50, 61, 30, 77, 46],
 'days_to_birth': [-18385, -22584, -11203, -28124, -16892]}

In [21]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [22]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'FEMALE', 'MALE', 'MALE', 'MALE']}

In [23]:
gender_col = 'gender'

In [24]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [25]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [26]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Acute Myeloid Leukemia,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-AB-2803-03,1,61,0,-8.843392,-3.942326,1.499765,0.300928,-0.393578,-0.82731,-0.699094,...,1.281922,-1.587117,-2.862233,-1.28139,-1.957877,-4.604286,-2.067127,0.321205,1.445867,-1.393012
TCGA-AB-2805-03,1,77,1,-0.194692,-4.815526,-0.531035,-0.609572,-0.518078,0.63379,-0.269494,...,-0.748878,0.001683,0.979467,-1.28139,-2.255877,-8.518386,-4.521027,-0.242595,-1.038833,-1.893912
TCGA-AB-2806-03,1,46,1,-9.153392,-2.155826,-0.531035,-0.261972,-0.654778,-0.53231,0.511106,...,0.435822,-1.587117,-3.289933,-1.28139,-1.909977,-8.782086,-3.331227,-0.576095,0.829167,-1.598212
TCGA-AB-2807-03,1,68,0,-6.172792,-3.562326,-0.531035,0.161828,1.023422,-1.03681,0.185706,...,2.701422,-1.587117,-0.577133,-1.28139,-1.182077,-4.395086,-3.773427,-0.735695,0.271967,-0.755412
TCGA-AB-2808-03,1,23,1,-3.978192,-5.554726,0.831365,-0.212472,-0.436278,-0.35651,0.081306,...,-0.748878,-1.587117,-0.631233,-1.28139,-1.996077,-6.678186,-5.717627,-0.544695,1.173567,-1.763812


In [27]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 173 samples.


In [28]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Acute Myeloid Leukemia', the least common label is '1' with 173 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Acute Myeloid Leukemia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 44.0
  50% (Median): 58.0
  75%: 67.0
Min: 18
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 80 occurrences. This represents 46.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [29]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [30]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Acute-Myeloid-Leukemia/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "X-Linked-Lymphoproliferative-Syndrome"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE156309', 'GSE235356']

Repeat the below steps for all the accession numbers

In [12]:
cohort = accession_num = "GSE235356"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/X-Linked-Lymphoproliferative-Syndrome/GSE235356/GSE235356_family.soft.gz',
 '../../DATA/GEO/X-Linked-Lymphoproliferative-Syndrome/GSE235356/GSE235356_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [13]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Expression data of plasma cell from MGUS patients"
!Series_summary	"Multiple myeloma is preceded by monoclonal gammopathy of undetermined significance (MGUS). Serum markers are currently used to stratify MGUS patients into clinical risk groups. A molecular signature predicting MGUS progression has not been produced. We have explored the use of gene expression profiling to risk-stratify MGUS and developed an optimized signature based on large samples with long-term follow-up."
!Series_summary	"Microarray analysis of mRNA from MGUS patients with stable disease and MGUS patients that progressed to MM within 10 years, was used to define a molecular signature of MGUS risk."
!Series_overall_design	"Bone marrow plasma cells from MGUS patients were enriched by anti-CD138 immunomagnetic beads for RNA extraction and hybridization on Affymetrix microarrays. We sought to identify genetic differences in the bone marrow plasma cells of stable and progressive MGUS patients. Patients wh

In [17]:
clinical_data.head()

,!Sample_geo_accession,GSM7500908,GSM7500909,GSM7500910,GSM7500911,GSM7500912,GSM7500913,GSM7500914,GSM7500915,GSM7500916,...,GSM7501256,GSM7501257,GSM7501258,GSM7501259,GSM7501260,GSM7501261,GSM7501262,GSM7501263,GSM7501264,GSM7501265
0,!Sample_characteristics_ch1,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,...,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells,cell type: Bone marrow plasma cells
1,!Sample_characteristics_ch1,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,disease state: Stable MGUS,...,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS,disease state: Progressing MGUS


In [18]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell type: Bone marrow plasma cells'],
 1: ['disease state: Stable MGUS', 'disease state: Progressing MGUS']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [19]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'X-Linked Lymphoproliferative Syndrome\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'X-Linked Lymphoproliferative Syndrome\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where uniqu

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [20]:
is_gene_availabe = False
trait_row_id = 0
age_row_id = 0
gender_row_id = None

trait_type = 'binary'

In [21]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [12]:
def convert_trait(value):
    """
    Convert a given value to a binary data type indicating the presence (1) or absence (0) of thyroid cancer.
    Unknown or non-applicable cases are converted to None.

    Args:
    value (str): A value from the dataset representing a characteristic related to thyroid cancer.

    Returns:
    int or None: Binary representation of the presence or absence of thyroid cancer.
    """
    # Define the traits indicating the presence of thyroid cancer
    cancer_indicators = [
        "histological variant: Classical", "histological variant: Follicular",
        "tissue type: Primary tumor", "tissue type: synchronous lymph node metastasis",
        "tissue type: lymph node metastasis post RAI", "tissue type: lymph node metastasis_1 post RAI",
        "tissue type: lymph node metastasis_2 post RAI", "patient rai responce: Refractory",
        "disease: Persistence"
    ]

    # Define the traits indicating the absence of thyroid cancer
    non_cancer_indicators = [
        "histological variant: non-neoplastic thyroid", "tissue type: non-neoplastic thyroid",
        "patient rai responce: Avid", "disease: Remission"
    ]

    # Convert to binary representation
    if value in cancer_indicators:
        return 1
    elif value in non_cancer_indicators:
        return 0
    else:
        return None

def convert_age(age_string):
    """
    Converts an age string in the format 'age: [number]' to an integer.
    If the format is incorrect or age is not specified, returns None.

    Parameters:
    age_string (str): The age string to be converted.

    Returns:
    int or None: The converted age as an integer, or None if conversion is not possible.
    """
    try:
        # Split the string and extract the age part
        age = age_string.split(': ')[1]
        # Convert the age to an integer
        return int(age)
    except (IndexError, ValueError):
        # Return None if there's an error in conversion
        return None


# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(value):
    if 'Sex: M' in value:
        return 0  # Male
    elif 'Sex: F' in value:
        return 1  # Female
    else:
        return None


In [13]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM4567912,GSM4567913,GSM4567914,GSM4567915,GSM4567916,GSM4567917,GSM4567918,GSM4567919,GSM4567920,GSM4567921,...,GSM4567954,GSM4567955,GSM4567956,GSM4567957,GSM4567958,GSM4567959,GSM4567960,GSM4567961,GSM4567962,GSM4567963
Thyroid Cancer,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [14]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM4567912,GSM4567913,GSM4567914,GSM4567915,GSM4567916,GSM4567917,GSM4567918,GSM4567919,GSM4567920,...,GSM4567954,GSM4567955,GSM4567956,GSM4567957,GSM4567958,GSM4567959,GSM4567960,GSM4567961,GSM4567962,GSM4567963
0,23064070,7.916905,8.238463,7.102249,8.409740,7.796908,8.543842,8.119806,7.974991,7.717500,...,8.245977,8.089905,8.276759,8.103044,7.939056,8.047937,7.703067,8.110620,8.000643,8.467071
1,23064071,7.669090,7.251441,7.229114,7.658347,7.432242,7.720535,7.630414,7.614341,7.097189,...,7.273556,7.479053,7.698748,7.832119,7.356233,7.221931,7.365635,7.472226,7.372839,7.474417
2,23064072,6.027989,6.798568,6.699349,6.587692,6.631769,6.568041,6.578532,6.529071,6.677197,...,6.922347,6.443426,6.269851,6.472525,6.766239,5.672337,6.107567,6.518080,6.566371,6.743142
3,23064073,7.424980,6.540915,6.219601,6.704671,6.683149,6.884614,7.067201,6.805651,7.188896,...,7.316303,7.165843,7.225258,7.173832,7.048203,7.448294,6.912840,7.004647,6.531787,7.021519
4,23064074,5.166980,5.025235,4.856050,5.555915,4.553328,5.291337,4.834381,4.985611,5.129950,...,5.043767,5.055717,4.888551,5.172946,4.823698,4.759997,4.445556,5.071252,4.605599,5.064432


In [15]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['23064070',
 '23064071',
 '23064072',
 '23064073',
 '23064074',
 '23064075',
 '23064076',
 '23064077',
 '23064078',
 '23064079',
 '23064080',
 '23064081',
 '23064083',
 '23064084',
 '23064085',
 '23064086',
 '23064087',
 '23064088',
 '23064089',
 '23064090']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['23064070', '23064071', '23064072', '23064073', '23064074', '23064075', '23064076', '23064077', '23064078', '23064079', '23064080', '23064081', '23064083', '23064084', '23064085', '23064086', '23064087', '23064088', '23064089', '23064090']\n"

If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
gene_annotation_summary

{'ID': ['TC0100006437.hg.1',
  'TC0100006476.hg.1',
  'TC0100006479.hg.1',
  'TC0100006480.hg.1',
  'TC0100006483.hg.1'],
 'probeset_id': ['TC0100006437.hg.1',
  'TC0100006476.hg.1',
  'TC0100006479.hg.1',
  'TC0100006480.hg.1',
  'TC0100006483.hg.1'],
 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'],
 'strand': ['+', '+', '+', '+', '+'],
 'start': ['69091', '924880', '960587', '966497', '1001138'],
 'stop': ['70008', '944581', '965719', '975865', '1014541'],
 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0],
 'category': ['main', 'main', 'main', 'main', 'main'],
 'SPOT_ID': ['Coding',
  'Multiple_Complex',
  'Multiple_Complex',
  'Multiple_Complex',
  'Multiple_Complex'],
 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // ch

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_name_key = key_name1
gene_name_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_name_key = key_name1\ngene_name_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Com

In [81]:
if requires_gene_mapping:
    probe_id_key = 'probeset_id'
    gene_symb_key = ''
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'ID' not found in columns."

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [57]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [58]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [35]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [36]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [37]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [160]:
merged_data.head()

,Bladder Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
0,1,63,1,0.221008,4.428374,-0.531035,0.720328,-0.408078,-0.39141,0.048706,...,-0.086078,-0.128017,0.094667,3.01571,0.049323,-0.811686,-0.418127,0.095905,1.301467,0.351488
1,1,66,1,1.334508,-2.541326,-0.531035,0.490728,0.523922,-0.59311,0.178806,...,0.124822,-1.085217,-2.360533,1.65561,-0.402877,-1.218686,-0.135527,0.417805,0.588167,-0.353612
2,1,69,1,1.173308,-5.554726,-0.531035,1.647728,0.518622,0.02829,0.800406,...,-0.748878,0.498683,-0.768233,-0.33489,-0.685077,-1.435286,0.392573,-0.115395,-1.106033,0.085188
3,1,59,0,1.090208,0.971574,-0.531035,0.256228,0.586622,-0.73021,0.636006,...,0.214422,-1.587117,-1.082733,1.90011,0.422823,-1.232086,-1.583627,0.596105,-0.924533,-0.322112
4,1,83,1,0.955008,-1.481926,0.115365,-0.039972,0.746322,-0.59151,0.365906,...,-0.748878,-1.587117,-0.136933,1.87471,0.027523,-0.881086,-0.462527,-0.186195,-2.369433,0.460088


In [161]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [162]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [163]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [164]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [165]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 76.94% ± 5.40%


In [166]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [167]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0073
Found 48 significant genes associated with the trait 'Bladder Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
  Variable  Coefficient  corrected_p_value
     MMP27    -0.067101       5.961268e-15
    GPR112    -0.066292       1.110854e-13
    CLEC3B    -0.084766       3.380228e-05
     LMX1A    -0.050085       3.380228e-05
   FAM180B    -0.053702       7.087183e-05
   COL10A1     0.081640       9.953343e-05
  ITGB1BP3    -0.043691       1.183858e-04
     TAAR6    -0.037012       1.183858e-04
    OLFML1    -0.096401       1.747172e-04
    VSTM2A    -0.045648       2.089221e-04
      ESM1     0.059578       3.629186e-04
    KBTBD5    -0.035954       3.821612e-04
      CST2     0.042439       3.821612e-04
NCRNA00110    -0.035798       5.219001e-04
    FBXL21    -0.042624       5.369488e-04
    OR2T27    -0.035769       1.088104e-03
     INHBA     0.065175       1.088104e-03
       F1

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
